In [1]:
# default_exp word_embeddings

In [18]:
# export

import os
import sys
import tqdm

from sklearn import feature_extraction, metrics
import numpy as np
import pandas as pd

import mlutil.parallel

import nltk
from gensim.models import doc2vec

import gensim
from github_search import repository_descriptions, neptune_util

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
all_codesearch_df = repository_descriptions.get_all_codesearch_df("data/python")
repos_descriptions_df = repository_descriptions.load_pypi_repo_descriptions()

In [5]:
codesearch_df = all_codesearch_df[
    all_codesearch_df["repo"].isin(repos_descriptions_df["repo"])
]

In [6]:
train_codesearch_df = codesearch_df[codesearch_df["partition"] == "train"]

In [7]:
train_comments = train_codesearch_df["docstring"]

In [8]:
repo_idx = train_codesearch_df["repo"].astype("category").cat.codes
train_codesearch_df["repo_idx"] = repo_idx

<ipython-input-8-49067e2b050c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_codesearch_df['repo_idx'] = repo_idx


In [9]:
train_descriptions = repos_descriptions_df.merge(
    train_codesearch_df[["repo", "repo_idx"]], on="repo"
).drop_duplicates("repo")

In [10]:
train_descriptions.shape

(8036, 3)

In [11]:
def make_document(words, doc_label):
    return doc2vec.TaggedDocument(nltk.word_tokenize(words.lower()), [doc_label])

In [12]:
description_documents = train_descriptions.apply(
    lambda x: make_document(x.pypi_description, x.repo_idx), axis=1
)

In [13]:
docstring_documents = train_codesearch_df.apply(
    lambda x: make_document(x.docstring, x.repo_idx), axis=1
)

In [14]:
corpus = pd.concat([docstring_documents, description_documents])

In [19]:
corpus.iloc[0]

TaggedDocument(words=['return', 'either', 'the', 'full', 'or', 'truncated', 'version', 'of', 'a', 'qiime-formatted', 'taxonomy', 'string', '.', ':', 'type', 'p', ':', 'str', ':', 'param', 'p', ':', 'a', 'qiime-formatted', 'taxonomy', 'string', ':', 'k__foo', ';', 'p__bar', ';', '...', ':', 'type', 'level', ':', 'str', ':', 'param', 'level', ':', 'the', 'different', 'level', 'of', 'identification', 'are', 'kingdom', '(', 'k', ')', ',', 'phylum', '(', 'p', ')', ',', 'class', '(', 'c', ')', ',', 'order', '(', 'o', ')', ',', 'family', '(', 'f', ')', ',', 'genus', '(', 'g', ')', 'and', 'species', '(', 's', ')', '.', 'if', 'level', 'is', 'not', 'provided', ',', 'the', 'default', 'level', 'of', 'identification', 'is', 'species', '.', ':', 'rtype', ':', 'str', ':', 'return', ':', 'a', 'qiime-formatted', 'taxonomy', 'string', 'up', 'to', 'the', 'classification', 'given', 'by', 'param', 'level', '.'], tags=[6911])

In [ ]:
neptune_util.init_neptune_project()

In [16]:
%%time
doc2vec_model = gensim.models.Doc2Vec(list(corpus))

CPU times: user 1min 38s, sys: 5.53 s, total: 1min 43s
Wall time: 1min


In [29]:
model_path = "/tmp/doc2vec_description_comments.bin"
experiment = neptune_util.init_neptune_experiment("doc2vec_description_comments", {})
doc2vec_model.save("/tmp/doc2vec_description_comments.bin")

https://ui.neptune.ai/lambdaofgod/github-search/e/GIT-17


In [30]:
import neptune

experiment.log_artifact(model_path)

In [32]:
experiment.stop()

In [17]:
doc2vec_model.infer_vector(description_documents[0].words)

array([ 2.00192686e-02,  1.56676266e-02, -1.35306463e-01, -4.47537005e-02,
       -3.62644009e-02,  1.05705887e-01,  2.35368926e-02,  3.56989391e-02,
        4.08363305e-02, -3.99746746e-02, -1.32620454e-01,  9.42355841e-02,
       -6.57281429e-02, -2.87903585e-02,  2.13103276e-02,  1.01808466e-01,
        6.84939176e-02, -2.48240046e-02,  2.54330356e-02,  8.47146362e-02,
       -8.85893330e-02, -7.15414062e-03, -1.19904682e-01, -2.11598095e-03,
       -5.45727760e-02,  1.05122747e-02,  1.01896994e-01, -1.17142778e-02,
        4.19061743e-02, -1.71306543e-02, -9.79323238e-02,  5.05158789e-02,
        4.28522825e-02, -2.49784309e-02, -6.82219258e-03,  1.12462444e-02,
        6.42626211e-02, -5.56820631e-02,  7.37288222e-02, -6.63584322e-02,
       -1.45307155e-02, -5.56058809e-02, -4.82847244e-02,  2.23817546e-02,
        4.67176400e-02, -9.42926574e-03,  7.68230408e-02, -4.30113412e-02,
        6.67387545e-02,  4.10580188e-02,  4.72838767e-02, -1.22534730e-01,
       -9.60843638e-02,  